<a href="https://colab.research.google.com/github/LucasMatuszewski/Python-colab-notebooks/blob/main/CrewAI_Chroma_RAG_NewsAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CrewAI RAG with Chroma DB and NewsAPI
Based on [this YouTube video](https://www.youtube.com/watch?v=77xSbC-9yn4)

## Mount Google Drive

for Data / Dependencies Persistance we can save them in our Google drive
Either as Virtual Env (for separation of packages if we install them for many different tools) or just by mounting Drive.

More: https://stackoverflow.com/questions/55253498/how-do-i-install-a-library-permanently-in-colab

In [ ]:
# PERSISTENCE of dependencies!
# to don't have to install dependencies each time we start a runtime, we can save files to Google Drive:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
gdrive = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', gdrive)
sys.path.insert(0,gdrive) # or append(nb_path)

# and then we can install like this:
# !pip install --target=$gdrive crewai


# ALTERNATIVE: Virtual Environment:

# Install virtualenv:
# !pip install virtualenv

# # Mount Google Drive:
# from google.colab import drive
# drive.mount("/content/drive")

# # Activate Virtual Environment and Install required libraries:
# !virtualenv /content/drive/MyDrive/vir_env
# !source /content/drive/MyDrive/vir_env/bin/activate; pip install numpy

# # Adding the Virtual Environment to sys.path:
# import sys
# sys.path.append("/content/drive/MyDrive/vir_env/lib/python3.10/site-packages")



## Install dependencies

In [1]:

# Install only once. Tomorrow, you can skip this (it will be loaded from Google Drive)

!pip install --target=$gdrive crewai
!pip install --target=$gdrive 'crewai[tools]'
!pip install --target=$gdrive duckduckgo-search langchain-community langchain-openai langchain-mistralai requests chromadb

MessageError: Error: credential propagation was unsuccessful

## Get API Keys and set all ENV vars


In [ ]:
import os
from google.colab import userdata

OpenAI_api_key = userdata.get("OpenAIkey")
os.environ['OPENAI_API_KEY'] = OpenAI_api_key
Mistral_api_key = userdata.get("MistralKey")
os.environ['MISTRAL_API_KEY'] = Mistral_api_key
news_api_key = userdata.get('NewsAPI')
os.environ['NEWS_API_KEY'] = news_api_key

# LangChain tracing LLM usage and bugs with LangSmith: https://python.langchain.com/docs/langsmith/walkthrough
LangChain_api_key = userdata.get('LangChain')
os.environ['LANGCHAIN_API_KEY'] = LangChain_api_key
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ["LANGCHAIN_PROJECT"] = "NewsAPI RAG"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

# if secretName == "MistralKey":
#   os.environ['OPENAI_API_BASE']="https://api.mistral.ai/v1"
#   os.environ['OPENAI_MODEL_NAME']="mistral-small"
# else:
os.environ['OPENAI_API_BASE']="https://api.openai.com/v1"
os.environ["OPENAI_MODEL_NAME"]="gpt-4-0125-preview"

## Import dependencies

In [ ]:
import requests
from langchain_openai import ChatOpenAI # tool to interact with OpenAI chat-style LLM's (like ChatGPT)
from langchain_mistralai.chat_models import ChatMistralAI # tool to interact with Mistral chat-style LLM's
from langchain_core.retrievers import BaseRetriever # Abstract base class for a Document retrieval system (search for documents by queries): https://api.python.langchain.com/en/latest/retrievers/langchain_core.retrievers.BaseRetriever.html
from langchain_community.document_loaders import WebBaseLoader # 1. Load the content of specific URL
from langchain.text_splitter import RecursiveCharacterTextSplitter # 2. Split web conent into chunks
from langchain_openai import OpenAIEmbeddings # 3. Convert splited web content to Embedings with OpenAI embedding models by API = costs money: https://platform.openai.com/docs/guides/embeddings
from langchain_community.vectorstores import Chroma # 4. Store embedings in Chroma DB
from langchain.tools import tool # build a pipeline, chain of different tools. Each tool has a description. Agent uses the description to choose the right tool for the job.
from langchain_community.tools import DuckDuckGoSearchRun # performs a web search in DuckDuckGo

## Set our TOOLs

In [ ]:
embedding_function = OpenAIEmbeddings()
llm_provider = "OpenAI" # @param ["Mistral", "OpenAI"] {type:"string"}
if llm_provider == "OpenAI":
  llm = ChatOpenAI(model="gpt-4-turbo-preview")
elif llm_provider == "Mistral":
  mistral_api_key = os.getenv('MISTRAL_API_KEY')
  llm = ChatMistralAI(mistral_api_key=mistral_api_key)
else:
  print("choose LLM Provider")

# Tool 1 : Save the news articles in a Chroma database
class SearchNewsDB:
  @tool("News DB Tool")
  def news (query: str):
    """Fetch news articles and process their contents."""
    NEWS_API_KEY = os.getenv('NEWS_API_KEY') # Fetch API key from environnent var
    base_url = "https://newsapi.org/v2/everything"
    # FREE VERSION OF NEWS API provides:
    # - articles 1 day old (not from today) and not older than 1 month
    # - 100 requests per day

    params = {
      'q': query,
      'sortBy': 'publishedAt',
      'apiKey': NEWS_API_KEY,
      ' language': 'en',
      'ageSize': 5,
    }

    response = requests.get(base_url, params) # Fetch list of news articles returned for our query
    if response.status_code != 200:
      return "Failed to retrieve news."

    articles = response.json().get('articles', [])
    all_splits = [] # all splits from all fetched articles
    for article in articles:
      # Assuming WebBaseLoader can handle a list of URLS
      loader = WebBaseLoader(article['url']) # load the content of specific article from internet by it's URL
      docs = loader.load() # execute the loading

      # This text splitter is the recommended one for generic text (split by: paragraphs, sentences, words)
      text_splitter = RecursiveCharacterTextSplitter(
          # DOCS: https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter
          # Set a really small chunk size, just to show.
          chunk_size=1000,
          chunk_overlap=200,
          # length_function=len,
          # is_separator_regex=False,
      )
      splits = text_splitter.split_documents(docs) # OR: create_documents([docs]), split_text(docs)[:2]
      all_splits.extend(splits)

    # Index the accumulated content splits if there are any
    if all_splits:
      # Chrome is Vector DB, same as Redis (key:value pairs)
      # DOCS: https://python.langchain.com/docs/integrations/vectorstores/chroma
      vectorstore = Chroma.from_documents(all_splits, embedding=embedding_function, persist_directory="./chroma_db")
      retriever = vectorstore.similarity_search(query) # search all splits of all articles by similarity to our query
      return retriever
    else:
      return "No content available for processing."

# Tool 2 : Get the news articles from the database
class GetNews:
  @tool("Get News Tool")
  def news(query: str) -> str:
    """Search Chroma DB for relevant news information based on a query."""
    vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=embedding_function)
    retriever = vectorstore.similarity_search(query)
    return retriever

# Tool 3 : Search for news articles on the web
search_tool = DuckDuckGoSearchRun()

# Tool 4 : sanitize query to make safe filename
import re
class SanitizeFilename:
  @tool("Sanitize Filename")
  def sanitize(query):
      """Removes unsafe characters and replaces spaces with hyphens."""
      # Characters to keep: letters, numbers, hyphens, underscores
      safe_chars = r" -_a-zA-Z0-9"
      # Regex to match anything NOT in your safe characters
      unsafe_pattern = re.compile(rf"[^{safe_chars}]")
      # Replace unsafe parts with an empty string
      sanitized_query = unsafe_pattern.sub("", query)
      # Replace spaces with hyphens
      return sanitized_query.replace(" ", "-")


## Create Agents

In [ ]:
from crewai import Agent

default_max_rpm=40

# 2. Creating Agents
news_search_agent = Agent(
  role='News Searcher',
  goal='Generate key points for each news article from the latest news',
  backstory="""Expert in analysing and generating key points from news content
    for quick summaries of the most important data and information.
    Pasionate about finding inspiring, surprising and numeric data and creating insights based on them.
    You always add an exact link to the source of the information.
    You like to write a well structured text in Markdown""",
  tools=[SearchNewsDB().news],
  allow_delegation=True,
  verbose=True,
  # max_iter=5,
  max_rpm=default_max_rpm,
  memory=True,
  llm=llm
)

writer_agent = Agent(
  role='Writer',
  goal="""Identify all the topics received. Use the Get News Tool to verify the each topic
    and use search_tool to verify facts and get more information. Write engaging articles based on all data.""",
  backstory="""Expert in crafting engaging narratives from complex information.
    You always add an exact link to the source of the information.
    You like to write a well structured text in Markdown""",
  tools=[GetNews().news, search_tool], # RAG - retriving data from embedings
  allow_delegation=True,
  verbose=True,
  # max_iter=5,
  max_rpm=default_max_rpm,
  memory=True,
  llm=llm
)

## Create tasks

In [ ]:
from crewai import Task
from datetime import datetime

# 3. Creating Tasks
search_phrase="microlearning"  # @param {type:"string"}

clean_phrase = SanitizeFilename().sanitize(search_phrase)

# Get current datetime
now = datetime.now()
formatted_datetime = now.strftime("%Y-%m-%d-%H-%M")

news_search_task = Task(
  description=f'Search for {search_phrase} and create key points for each news.',
  agent=news_search_agent,
  tools=[SearchNewsDB().news],
  output_file=f"{clean_phrase}--search-{formatted_datetime}.md",
  expected_output='A Markdown formatted text with key points for each news and with exact links to the source in markdown format.'
)

writer_task = Task(
  description="""
  Go step by step.
  Identify all the topics received.
  Use the Get News Tool to verify the each topic by going through one by one.
  Use the Search tool to search for information on each topic one by one.
  Go through every topic and write an in-depth summary of the information retrieved.
  Don't skip any topic.
  Add 3 short tags for most important information in a form: #tag-1-name #tag-2-name
  """,
  agent=writer_agent,
  context=[news_search_task],
  tools= [GetNews().news, search_tool],
  output_file=f"{clean_phrase}--summary-{formatted_datetime}.md",
  expected_output='A Markdown formatted text with in-depth summaries of the information retrieved and with exact link to the source in markedown format.'
)

## Create a Crew

In [ ]:
from crewai import Crew, Process
# 4. Creating Crew
news_crew = Crew(
  agents=[news_search_agent, writer_agent],
  tasks=[news_search_task, writer_task],
  process=Process.sequential, # we can test also with herarchical Process
  manager_llm=llm
)

## Run and print results

In [ ]:
result = news_crew.kickoff()
print(result)



> Entering new CrewAgentExecutor chain...
Thought: To generate key points from the latest news articles on microlearning, I need to fetch the relevant news articles first. I will use the News DB Tool to query news articles related to microlearning.

Action: News DB Tool

Action Input: {"query": "microlearning"}
 

[Document(page_content='What Is Microlearning?\nMicrolearning is a learning method that focuses on short, targeted bursts of information delivery. It typically involves breaking down more prominent topics into smaller, stand-alone modules that can be completed in just a few minutes. This approach allows for quick and efficient learning, making it ideal for the fast-paced nature of the digital age.\nMicrolearning often utilizes multimedia elements to engage learners and enhance the learning experience. It also employs various delivery methods such as mobile apps, emails, and Learning Management Systems (LMSs) to make learning accessible anytime, anywhere.\nThe Benefits Of Mi